# Propagation guidée

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import scipy.optimize
import scipy.signal
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy as unp


%matplotlib

Using matplotlib backend: QtAgg


Mesure de la vitesse de propagation de modes acoustique

In [4]:
#Données / Paramètres

inc_L = 1.0
L = ufloat(192, inc_L)*10**-2 #Longueur du tuyau (m)

inc_d = 0.1
d = ufloat(15.3, inc_d)*10**-3 #Diamètre du tuyau (m)

Lambda = 8.5*10**-3 #longueur d'onde du signal (m)

inc_dt_precis = 0.1
inc_dt_imprecis = 0.2
dt = unp.uarray([5.64, 6.15, 7.28], [inc_dt_precis, inc_dt_imprecis, inc_dt_precis])*10**-3 #Temps mesurés de propagation des modes (s)

c_mes = L/dt #Vitesse mesurée de propagation des modes (m/s)

#Calcul de la vitesse théorique de propagation des modes (m/s)

inc_c0 = 0.2
c0 = ufloat(344.2, inc_c0)

mu_11 = 1.84
mu_21 = 3.05
mu_12 = 3.83

lambda_c = np.array([np.pi*d/mu_11, np.pi*d/mu_21, np.pi*d/mu_12])
lambda_g = lambda_c*Lambda/unp.sqrt(lambda_c**2 - Lambda**2)


c_th = c0*Lambda/lambda_g #Vitesse théorique de propagation des modes

#Affichage  des vitesses normalisées au mode fondamental

print (c_mes/c_mes[0])
print (c_th/c0)

[1.0+/-0 0.9170731707317074+/-0.033968148296001734
 0.7747252747252746+/-0.01737623382701306]
[0.9455821579787812+/-0.0007318131188763198
 0.8420763009707187+/-0.0022579381806746702
 0.7357136640387805+/-0.004075233891967898]


In [9]:
#Mesure longueur des cables

c = 3*10**8 #vitesse de la lumière dans le vie (m/s)
c_cablecoax = ufloat(0.67*c, 0.01*c) 

inc_dt = 5
dt = unp.uarray([0, 510, 320, 510], [0, inc_dt, inc_dt, inc_dt])*10**-9 #temps entre un impulsion et sa réflexion (s)

L = c_cablecoax*dt/2 #longueur du cable (m)

#Mesure de l'atténuation du signal en fonction de la fréquence

L[3] = L[3]+L[2]+L[1] #On branche les cables en série
L[2] = L[2]+L[1]

inc_U = 0.04
U_500k = unp.uarray([2.70, 2.42, 2.34, 2.18], [inc_U, inc_U, inc_U, inc_U]) #Amplitude (crete à crete) (V)
U_1M = unp.uarray([2.72, 2.38, 2.28, 2.04], [inc_U, inc_U, inc_U, inc_U])
U_2M = unp.uarray([2.60, 2.30, 2.14, 1.82], [inc_U, inc_U, inc_U, inc_U])
U_5M = unp.uarray([2.66, 2.14, 1.90, 1.42], [inc_U, inc_U, inc_U, inc_U])


A_500k = 20*unp.log(U_500k/U_500k[0]) #Coefficient d'atténuation (dB)
A_1M = 20*unp.log(U_1M/U_1M[0])
A_2M = 20*unp.log(U_2M/U_2M[0])
A_5M = 20*unp.log(U_5M/U_5M[0])
print (A_1M)


[0.0+/-0 -2.6706278524904548+/-0.4466447820476744
 -3.5291287468311334+/-0.4578427621647704
 -5.753641449035618+/-0.49019607843137253]


In [6]:
#Fit de l'atténuation

def func(x, a):
    return a*x

popt500, pcov500 = curve_fit(func, unp.nominal_values(L), unp.nominal_values(A_500k))
popt1, pcov1 = curve_fit(func, unp.nominal_values(L), unp.nominal_values(A_1M))
popt2, pcov2 = curve_fit(func, unp.nominal_values(L), unp.nominal_values(A_2M))
popt5, pcov5 = curve_fit(func, unp.nominal_values(L), unp.nominal_values(A_5M))

delta_pcov500 = np.sqrt(pcov500.diagonal(0))
delta_pcov500 = delta_pcov500[0]

delta_pcov1 = np.sqrt(pcov1.diagonal(0))
delta_pcov1 = delta_pcov1[0]

delta_pcov2 = np.sqrt(pcov2.diagonal(0))
delta_pcov2 = delta_pcov2[0]

delta_pcov5 = np.sqrt(pcov5.diagonal(0))
delta_pcov5 = delta_pcov5[0]


In [7]:
#Tracer les coefficients d'atténuations en fonction de la longueur du cable

plt.xlabel('Longueur du cable (en m)')
plt.ylabel('Atténuation (en dB)')

plt.plot(unp.nominal_values(L), unp.nominal_values(A_500k), 'y*')
plt.errorbar(unp.nominal_values(L), unp.nominal_values(A_500k), unp.std_devs(A_500k), unp.std_devs(L), 'yo')

plt.plot(unp.nominal_values(L), unp.nominal_values(A_1M), 'g*')
plt.errorbar(unp.nominal_values(L), unp.nominal_values(A_1M), unp.std_devs(A_1M), unp.std_devs(L), 'go')


plt.plot(unp.nominal_values(L), unp.nominal_values(A_2M), 'r*')
plt.errorbar(unp.nominal_values(L), unp.nominal_values(A_2M), unp.std_devs(A_2M), unp.std_devs(L), 'ro')


plt.plot(unp.nominal_values(L), unp.nominal_values(A_5M), 'b*')
plt.errorbar(unp.nominal_values(L), unp.nominal_values(A_5M), unp.std_devs(A_5M), unp.std_devs(L), 'bo')


plt.plot(unp.nominal_values(L), func(unp.nominal_values(L), *popt500), 'y')
plt.plot(unp.nominal_values(L), func(unp.nominal_values(L), *popt1), 'g') 
plt.plot(unp.nominal_values(L), func(unp.nominal_values(L), *popt2), 'r') 
plt.plot(unp.nominal_values(L), func(unp.nominal_values(L), *popt5), 'b') 

print ("L'atténuation est de (", 100*popt500[0], "+-", delta_pcov500*100, ")dB/100m à 500kHz, de (", 100*popt1[0], "+-", delta_pcov1*100, ")dB/100m à 1MHz, de (", 100*popt2[0], "+-", delta_pcov2*100, ")dB/100m à 2MHz et de (", 100*popt5[0], "+-", delta_pcov5*100, ")dB/100m à 5MHz.")



L'atténuation est de ( -3.344587996843001 +- 0.18418572647312287 )dB/100m à 500kHz, de ( -4.350859780976422 +- 0.16090596444641359 )dB/100m à 1MHz, de ( -5.090609551665548 +- 0.16485819901345644 )dB/100m à 2MHz et de ( -8.927773163637731 +- 0.3191672924514074 )dB/100m à 5MHz.
